In [7]:
import pandas_datareader as data
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import yfinance as yf
%matplotlib inline

**Make larger inputs (intra-day)**

In [75]:
intraday_data = yf.download(tickers="MSFT",
                            period="2y",
                            interval="1h",
                            auto_adjust=True)
intraday_data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
2020-05-14 09:30:00-04:00,177.539993,178.889999,175.735703,175.789993,8841683
2020-05-14 10:30:00-04:00,175.800003,177.940002,175.679993,177.260101,6053059
2020-05-14 11:30:00-04:00,177.279999,178.169998,176.000000,176.830093,4640860
2020-05-14 12:30:00-04:00,176.860001,178.154999,176.449997,178.109894,3232095
2020-05-14 13:30:00-04:00,178.100006,178.320007,177.190002,178.219696,2931715


In [76]:
intraday_data.axes[0][0]

Timestamp('2020-05-14 09:30:00-0400', tz='America/New_York')

In [77]:
sequence_opens = intraday_data.values[: , 0]

In [78]:
sequence_opens.shape

(3526,)

In [66]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [93]:
# choose a number of time steps
n_steps = 64
# split into samples
X, y = split_sequence(sequence_opens, n_steps)

In [94]:
X.shape

(3462, 64)

In [19]:
pd.options.plotting.backend = "plotly"
intraday_data.plot(y=['Open'])

**Split and flip data randomly**

**Wavenet**

**Test and train evaluation**

**Scatter plots (correlation)**